In [118]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from lib import load_data
%matplotlib inline

In [208]:
users_df, costs_df, relations_df, visits_df, orders_df = load_data()

/Users/v/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


## Задание №1:	
## С помощью инструментов R или Python посчитать следующие метрики:

### ROI для 0-го, 6-го 14-го и 30-го дней жизни пользователя на сайте в разрезе маркетинговых каналов (chanel из relations).

#### Join Users with Realtions to get chanels per user and Costs to get cost per user
#### There are many records in Users that are dropped due to the missing Costs

In [209]:
print('Number of users in the table: {}'.format(len(users_df)))
users_df = users_df.merge(relations_df, how='inner', on=['name', 'id_partner'])
print('Number of users after merge with Relations: {}'.format(len(users_df)))

users_costs_df = users_df.merge(costs_df, how='inner', left_on=['name', 'id_partner', 'Reg_date'], 
                                        right_on=['campaign', 'id_partner', 'date_created'])
print('Number of users after merge with Costs: {}'.format(len(users_costs_df)))
users_df.set_index('id', inplace=True)
users_costs_df.set_index('id', inplace=True)

Number of users in the table: 2524443
Number of users after merge with Relations: 2363808
Number of users after merge with Costs: 1019797


In [212]:
users_costs_df['cost_per_user'] = users_costs_df.groupby(['name', 'id_partner', 'Reg_date'])['Costs'].transform(lambda x: np.array(x)[0]/len(x))
users_costs_df.head(20)

,Reg_date,id_partner,name,chanel,Costs,campaign,date_created,cost_per_user
id,,,,,,,,
7043369,2017-10-25,Clickdealer,-1,Clickdealer,10.0,-1,2017-10-25,10.0
6700130,2017-10-02,Advidi_email,-1,Advidi_email,20.0,-1,2017-10-02,10.0
6700181,2017-10-02,Advidi_email,-1,Advidi_email,20.0,-1,2017-10-02,10.0
5656121,2017-08-04,adsimilis_signup,-1,Adsimilis,9.0,-1,2017-08-04,9.0
5829186,2017-08-14,adsimilis_signup,-1,Adsimilis,9.0,-1,2017-08-14,9.0
7064570,2017-10-27,Adsimilis_internal,-1,Adsimilis,12.0,-1,2017-10-27,12.0
7064603,2017-10-27,mob_Adsimilis_internal,-1,Adsimilis,5.0,-1,2017-10-27,2.5
7064613,2017-10-27,mob_Adsimilis_internal,-1,Adsimilis,5.0,-1,2017-10-27,2.5
6557553,2017-09-25,Shift44,-1,Shift44,62.5,-1,2017-09-25,12.5


### Verify cost_per_user

In [230]:
regdate = np.datetime64('2017-09-25')
id_partner = 'Shift44'
name = '-1'
def verify_cost_per_day(regdate, id_partner, name):
    num_of_users_one_day_one_campaign = len(users_costs_df[(users_costs_df['Reg_date'] == regdate) & 
            (users_costs_df['id_partner'] == id_partner) &
            (users_costs_df['name'] == name)])
    print('Number of users for one campaign day: {}'.format(num_of_users_one_day_one_campaign))
    cost_for_one_day_that_campaign = costs_df[(costs_df['date_created'] == regdate) 
                                              & (costs_df['id_partner'] == id_partner) &
                                              (costs_df['campaign'] == name)]['Costs'].values[0]
    print('Cost per that campaign on user reg day: {}'.format(cost_for_one_day_that_campaign))
    cost_per_user_that_campaign = cost_for_one_day_that_campaign / num_of_users_one_day_one_campaign
    print('Cost per user on reg day for campaign: {}'.format(cost_per_user_that_campaign))

verify_cost_per_day(np.datetime64('2017-09-25'), 'Shift44', '-1')
verify_cost_per_day(np.datetime64('2017-10-25'), 'Clickdealer', '-1')

Number of users for one campaign day: 5
Cost per that campaign on user reg day: 62.5
Cost per user on reg day for campaign: 12.5
Number of users for one campaign day: 1
Cost per that campaign on user reg day: 10.0
Cost per user on reg day for campaign: 10.0


#### Join Orders with Users to get Registration date in Orders

In [234]:
orders_df = orders_df.merge(users_df['Reg_date'].to_frame(), how='inner', left_on='id_user', right_index=True)
orders_df.head()

,id_user,Order Date,Amount,Order ID,Reg_date
0,4618612,2017-08-17 02:08:40,2.6312,23732390,2017-06-01
1,4618612,2017-08-17 02:29:26,17.5912,23732304,2017-06-01
2,4618645,2017-06-01 04:39:41,8.7912,12441805,2017-06-01
3,4618815,2017-06-01 02:11:01,2.6312,12439685,2017-06-01
4,4618982,2017-10-04 03:43:24,8.7912,25568622,2017-06-01


In [236]:
def roi_on_day(day, users_df, orders_user_reg_df):
    users_ids = users_df[users_df['Reg_date'] < np.max(users_df['Reg_date']) + np.timedelta64(-day - 1, 'D')].index
    orders_for_roi_mask = orders_user_reg_df['Order Date'] < orders_user_reg_df['Reg_date'] + np.timedelta64(day + 1, 'D')
    orders_for_roi_mask = orders_for_roi_mask & orders_user_reg_df['id_user'].isin(users_ids)
    cost_order_amount_per_user_df = orders_user_reg_df[orders_for_roi_mask].groupby('id_user').agg(
        {'Amount': 'sum'})
    print(cost_order_amount_per_user_df.head())
    order_amount_day_label = 'order_amount_day_' + str(day)
    users_df[order_amount_day_label] = cost_order_amount_per_user_df['Amount']
    users_df[order_amount_day_label].fillna(0, inplace=True)
    
    cost_amout_per_channel_df = users_df.groupby('chanel').agg({'cost_per_user': 'sum', order_amount_day_label: 'sum'})
    roi = 100 * (cost_amout_per_channel_df[order_amount_day_label] 
        - cost_amout_per_channel_df['cost_per_user']) / cost_amout_per_channel_df['cost_per_user']
    return roi

## Answer

In [237]:
roi = pd.DataFrame({'roi_0': roi_on_day(0, users_costs_df, orders_df)})
# roi['roi_14'] =  roi_on_day(14, users_df, orders_df)
# roi['roi_30'] =  roi_on_day(30, users_df, orders_df)
# roi

          Amount
id_user         
4620243  64.2224
4621577   8.7912
4625930   2.6312
4628666   2.6312
4628943   2.6312


In [146]:
len(users_df)

1019797

## Стоимость за пользователя, который совершил свою первую оплату в первые 14, 30 и 60 дней жизни пользователя на сайте в разрезе маркетинговых партнеров (id_partner).

In [125]:
def user_cost_paid_on_day(day, users_df, first_orders):
    users_mask = np.isin(users_df.index.values, first_orders.index.values)
    users_mask &= users_df['Reg_date'] < np.max(users_df['Reg_date']) + np.timedelta64(-day - 1, 'D')
    user_cost_per_partner = users_df[users_mask].groupby('id_partner').agg({'cost_per_user': 'mean'})
    return user_cost_per_partner

In [126]:
first_orders = orders_df.groupby('id_user').agg({'Order Date': lambda x: np.array(x.sort_values())[0]})
print('Average cost paid for user per partners in first {} days of users lifetime:'.format(14))
print(user_cost_paid_on_day(14, users_df, first_orders))

Average cost paid for user per partners in first 14 days of users lifetime:
                           cost_per_user
id_partner                              
10                              6.453358
1020                            3.301604
1020_mobile                     1.317024
1020n                           3.041687
14186                          20.762711
15                              4.724168
29                              5.363911
37                              3.942065
7                               2.213143
7click_email                    4.915420
9                               9.533036
ADC                             1.687500
Ad4date                         6.371220
Ad4date_mobile                  2.772727
Adsimilis                       7.002381
Adtrafico                       4.630768
Adtrafico_mobile                3.034021
Advidi                          6.907041
Affiliaxe_mobile                2.346079
Affiliaxe_non_adult_valid       5.953166
Beeclick              

In [127]:
print('Average cost paid for user per partners in first {} days of users lifetime:'.format(30))
print(user_cost_paid_on_day(30, users_df, first_orders))

Average cost paid for user per partners in first 30 days of users lifetime:
                           cost_per_user
id_partner                              
10                              6.453358
1020                            3.301604
1020_mobile                     1.317024
1020n                           3.041687
14186                          19.589086
15                              4.724168
29                              5.363911
37                              3.942065
7                               2.213143
7click_email                    4.883521
9                               6.629070
ADC                             1.687500
Ad4date                         6.371220
Ad4date_mobile                  2.772727
Adsimilis                       7.002381
Adtrafico                       4.630768
Adtrafico_mobile                3.034021
Advidi                          6.907041
Affiliaxe_mobile                2.346079
Affiliaxe_non_adult_valid       5.953166
Beeclick              

In [128]:
print('Average cost paid for user per partners in first {} days of users lifetime:'.format(60))
print(user_cost_paid_on_day(60, users_df, first_orders))

Average cost paid for user per partners in first 60 days of users lifetime:
                           cost_per_user
id_partner                              
10                              6.198840
1020                            3.301604
1020_mobile                     1.317024
1020n                           3.374733
14186                          22.184431
7                               2.279589
7click_email                    4.789082
9                               9.743534
ADC                             1.687500
Ad4date                         6.371220
Ad4date_mobile                  2.772727
Adsimilis                       7.002381
Adtrafico                       4.630768
Adtrafico_mobile                3.034021
Advidi                          6.856096
Affiliaxe_mobile                2.288005
Affiliaxe_non_adult_valid       5.846434
Beeclick                        8.359081
Beeclick_mobile                 3.970502
CPATrend_mobile                 3.583877
CPATrend_non_adult_val

### % пользователей, которые вернулись на сайт хотя бы раз в течении первой недели своей жизни в разрезе маркетинговых каналов (chanel из relations).

In [129]:
def pct_users_return_on_day(day, users_df):
    visits_mask = users_df['Visit_date'] < users_df['Reg_date'] + np.timedelta64(day + 1, 'D')
    visits_mask = visits_mask & ~users_df['Visit_date'].isnull()
    visited_mask_label = 'mask_visited_till_day_'+str(day)
    users_df[visited_mask_label] = visits_mask
    visited_till_day = users_df.groupby('chanel').agg(
        {visited_mask_label: lambda x: 100 * x.sum() / len(x)})
    return visited_till_day

In [130]:
first_visits_df = visits_df.sort_values(by='Visit_date').drop_duplicates('Visit_date', keep='first')
users_df = users_df.merge(first_visits_df, how='left', right_on='id_user', left_index=True)
visited_till_day = pct_users_return_on_day(7, users_df)
visited_till_day

,mask_visited_till_day_7
chanel,
10,37.717391
15,36.912752
29,40.339306
32,68.013468
37,39.158683
52,100.000000
7,34.394638
7click_email,29.928793
8,38.061617


#### Determine that Reg_date is not accounted as the first record in the Visit_date

In [131]:
# (first_visits_df['Visit_date'].dt.normalize() == first_visits_df['Reg_date'].dt.normalize()).value_counts()

### Количество логинов на пользователя в разрезе маркетинговых каналов (chanel из relations). 

In [132]:
visits_users_df = visits_df.merge(users_df['chanel'].to_frame(), how='inner', left_on='id_user', right_index=True)

In [133]:
zz = visits_users_df.groupby(['chanel', 'id_user']).agg({'id_user': 'count'}).groupby(level=0)

In [134]:
# len(visits_users_df.groupby(['chanel', 'id_user']).agg({'id_user': 'count'}).groupby(level=0))

In [138]:
users_df.index

Int64Index([19917371, 16187818, 16187818, 16187818, 26622367, 16187818,
            27133847, 16187818, 16187818, 26631612,
            ...
              979961,   979964,   979965,   979966, 23078376, 27462394,
             4701622, 16187818,  4701624,  4701625],
           dtype='int64', length=5338349)